<a href="https://colab.research.google.com/github/diviyajohndj/Drug-to-Drug-interaction-prediction/blob/main/drug_test_working(6may).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [220]:
repo_url= "https://github.com/diviyajohndj/Drug-to-Drug-interaction-prediction"
import os

%cd /content

repo_dir_path = os.path.abspath(os.path.join('.', os.path.basename(repo_url)))

!git clone {repo_url}
%cd {repo_dir_path}
!git pull

/content
fatal: destination path 'Drug-to-Drug-interaction-prediction' already exists and is not an empty directory.
/content/Drug-to-Drug-interaction-prediction
Already up to date.


In [221]:
#csv file read,write
import pandas as pd
#random, permutations
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import cross_val_score
from sklearn.model_selection  import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import gensim
import spacy
from spacy.symbols import nsubj, VERB
from spacy.symbols import det

#pipelines for model
nlp = spacy.load('en')

In [222]:
train_data_f ="train_binary_offsets.csv"
test_data_f ="test_binary_offsets.csv"
#input drug
idata_f="input.csv"


df_train = pd.read_csv(train_data_f, low_memory=False)
df_test = pd.read_csv(test_data_f, low_memory=False)
#input
idata=pd.read_csv(idata_f)

#same drug names filter
df_train = df_train[df_train['drug1'] != df_train['drug2']]
df_test = df_test[df_test['drug1'] != df_test['drug2']]
#input
idata=idata[idata['drug1']!=idata['drug2']]


In [223]:
idata.head()

,sentenceid,sentence_text,drug_pair,ddi_label,drug1,drug2,drug1offset,drug2offset,drugsinsent
0,DDI-DrugBank.d0.s1,Pharmacokinetic studies indicate that administ...,DDI-DrugBank.d0.s1.p2,0,diazepam,acamprosate,70-77,119-129,"disulfiram,diazepam,acamprosate,"
1,DDI-DrugBank.d0.s2,Co-administration of naltrexone with Acamprosa...,DDI-DrugBank.d0.s2.p0,1,naltrexone,Acamprosate,21-30,37-47,"naltrexone,Acamprosate,acamprosate,"


In [224]:
print("Length of train dataset: ", df_train.shape)
print("Number of drug paris with no affect on each other: ", df_train.loc[df_train['ddi_label'] == 0].shape)
print("Number of drug pairs that do affect each other: ", df_train.loc[df_train['ddi_label'] == 1].shape)

print("\nLength of test dataset: ", df_test.shape)
print("Number of drug paris with no affect on each other: ", df_test.loc[df_test['ddi_label'] == 0].shape)
print("Number of drug pairs that do affect each other: ", df_test.loc[df_test['ddi_label'] == 1].shape)

Length of train dataset:  (223, 9)
Number of drug paris with no affect on each other:  (217, 9)
Number of drug pairs that do affect each other:  (6, 9)

Length of test dataset:  (209, 9)
Number of drug paris with no affect on each other:  (183, 9)
Number of drug pairs that do affect each other:  (26, 9)


In [225]:
def create_train_dev_test(df):
    random_index = np.random.permutation(df.index)
    df_shuffled = df.loc[random_index]
    df_shuffled.reset_index(drop=True, inplace=True)
    rows, columns =  df_shuffled.shape
    train_size = round(rows*.2)
    dev_size   = round(rows*.4)
    df_train = df_shuffled.loc[:train_size]
    df_dev = df_shuffled.loc[train_size:dev_size+train_size].reset_index(drop=True)
    return df_train, df_dev

#random picking of no-interaction rows from datasets
df_train.loc[df_train['ddi_label'] == 0].head()
df_train_no_interact, df_dev = create_train_dev_test(df_train.loc[df_train['ddi_label'] == 0])

In [226]:
frames = [df_train_no_interact,df_train.loc[df_train['ddi_label'] == 1]]


df_train = pd.concat(frames)
df_train=df_train.sample(frac=1)
print(df_train.shape)

print("Number of drug paris with no affect on each other: ", df_train.loc[df_train['ddi_label'] == 0].shape)
print("Number of drug pairs that do affect each other: ", df_train.loc[df_train['ddi_label'] == 1].shape)

(50, 9)
Number of drug paris with no affect on each other:  (44, 9)
Number of drug pairs that do affect each other:  (6, 9)


In [227]:
import nltk
#tokenize sentences
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [228]:
triggers = []
with open("/content/Drug-to-Drug-interaction-prediction/triggers.txt") as f:
    for line in f:
        
        if line!="":
          triggers.append(line)
trigger_words = set(triggers)

negative_words = ['No', 'not', 'neither', 'without', 'lack', 'fail', 
                  'unable', 'abrogate', 'absence', 'prevent', 
                  'unlikely', 'unchanged', 'rarely']

negative_words = set(negative_words)

def tokenize(sentence_df):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = [sentence for sentence in sentence_df]
    words_in_sents = [nltk.word_tokenize(sent) for sent in sentences]
    return words_in_sents
#tokenize each sentence text 
words_in_sents = tokenize(df_train['sentence_text'])
''''
#embedding to word2vwec
ddi_embedding_model = gensim.models.Word2Vec(words_in_sents)
ddi_embedding_model.save('ddi.embedding')
new_model = gensim.models.Word2Vec.load('ddi.embedding')

'''





"'\n#embedding to word2vwec\nddi_embedding_model = gensim.models.Word2Vec(words_in_sents)\nddi_embedding_model.save('ddi.embedding')\nnew_model = gensim.models.Word2Vec.load('ddi.embedding')\n\n"

In [229]:
def negation_count(x, row):
    """counts total no, not and n't in the sentence """ 
    sent = row['sentence_text']
    count = 0
    for word in nltk.word_tokenize(sent):
        if word == 'not' or word == 'no' or word[-3:] == "n't":
            count += 1
    return count

def negation_count2(x, row):
    """ gives how many negative words in each segment"""
    count = 0
    drug1_i = row['drug1offset'].split('-')
    drug2_i = row['drug2offset'].split('-')
    sentence = row['sentence_text']
    before_count=0
    middle_count=0
    end_count=0
    first_phrase=""
    mid_phrase=""
    end_phrase=""
    try:
      first_phrase = sentence[0:int(drug1_i[0])].split()
      mid_phrase = sentence[int(float(drug1_i[-1]))+1:int(drug2_i[0])].split()
      end_phrase = sentence[int(float(drug2_i[-1]))+1:].split()
      before_count = len(set(first_phrase) & negative_words)
      middle_count = len(set(mid_phrase) & negative_words)
      end_count = len(set(end_phrase) & negative_words)

    except ValueError:
      print("ValueError")
    return before_count, middle_count, end_count
    

def trigger_count(x, row):
    """ total number of trigger words """
    sent = row['sentence_text']
    count = 0
    for word in nltk.word_tokenize(sent.lower()):
        if word in triggers:
            count += 1
    return count

def trigger_count2(x, row):
    ''' counts number of trigger words in each segment '''
    count = 0
    drug1_i = row['drug1offset'].split('-')
    drug2_i = row['drug2offset'].split('-')
    sentence = row['sentence_text']
    first_phrase =""
    mid_phrase=""
    end_phrase=""
    try:
      first_phrase = sentence[0:int(drug1_i[0])].split()
      mid_phrase = sentence[int(float(drug1_i[-1]))+1:int(drug2_i[0])].split()
      end_phrase = sentence[int(float(drug2_i[-1]))+1:].split()
    except ValueError:
      print("")
      #print("Value error in trigger_count2")
      #print("inside exception In trigger_count 2 first coubt, mid,end:",first_phrase,mid_phrase,end_phrase)
    before_count = len(set(first_phrase))# & trigger_words)
    middle_count = len(set(mid_phrase))# & trigger_words)
    end_count = len(set(end_phrase))# & trigger_words)
    #print("with and wthout excep: sentance:",sentence)
    #print("In trigger_count 2 first coubt, mid,end:",first_phrase,mid_phrase,end_phrase)
    #print("passing befor_count,middle_count,end_count",before_count,middle_count,end_count)
    return before_count, middle_count, end_count

def ultra_trigger(x, row):
    ''' counts ultra-trigger words '''
    sent = row['sentence_text']
    words = ['coadministration', 'concomitant', 'concomitantly']
    count = 0
    for word in words:
        if word in nltk.word_tokenize(sent.lower()):
            count += 1
    return count
        
def words_sep_by_and(x, row):
    ''' binary feature if drugs are seperated only by and, not used '''
    sent = row['sentence_text']
    d1 = row['drug1']
    d2 = row['drug2']
    sent = sent.replace(d1, 'drug1')
    sent = sent.replace(d2, 'drug2')
    words = nltk.word_tokenize(sent)
    if abs(words.index('drug1') - words.index('drug2')) < 3:
        if words[words.index('drug1') + words.index('drug2')/2] == 'and':
            return 1
    return 0

def key_phrase(x, row):
    ''' binary feature that checks for key phrases, listed below'''
    sent = row['sentence_text']
    d1 = row['drug1']
    d2 = row['drug2']
    sent = sent.replace(d1, 'drug1')
    sent = sent.replace(d2, 'drug2')
    phrases = ['concurrent administration of drug1 and drug2', 'drug1 concurrently with drug2',
            'co-administration of drug1 and drug2', 'coadministration of drug1 and drug2', 
               'concurrent use of drug1 and drug2',]
    for phrase in phrases:
        if phrase in sent.lower():
            return 1
    return 0
    
def num_words_between_drugs(x,row):
    ''' counts number of words between drug mentions '''
    drug1_i = row['drug1offset'].split('-')
    drug2_i = row['drug2offset'].split('-')
    sentence = row['sentence_text']
    pre_phrase=""
    try:
      pre_phrase = sentence[int(float(drug1_i[-1]))+1:int(float(drug2_i[0]))].split()
    except ValueError:
      print("eroor exception")
    return len(pre_phrase)

def num_drugs_between_drugs(x, row):
    ''' counts number of drugs between drug mentions '''
    drug1_i = row['drug1offset'].split('-')
    drug2_i = row['drug2offset'].split('-')
    sentence = row['sentence_text']
    pre_phrase=""
    try:

      pre_phrase = sentence[int(float(drug1_i[-1]))+1:int(float(drug2_i[0]))]
    except ValueError:
      print("error")
    list_of_drugs = row['drugsinsent'].split(",")[0:-1]
    drugs_in_bet = 0
    for drug in list_of_drugs:
        drug_split = drug.split()
        if(pre_phrase.find(drug_split[0]) != -1):
            drugs_in_bet += 1

    return drugs_in_bet

def verb_count(x, row):
    '''counts number of verbs in each segment '''
    pattern = r'''(?x)     
    ([A-Z]\.)+                       # abbrevations
    |\w+'\w+                         # contractions
    | \w+(?:(-|(\.\s))(\n)?\w+)*     # words w/ internal hyphens, extend to next line, and with periods like Mrs. Reed
    '''
    count = 0
    drug1_i = row['drug1offset'].split('-')
    drug2_i = row['drug2offset'].split('-')
    sentence = row['sentence_text']
    first_phrase=""
    mid_phrase=""
    end_phrase=""
    try:
      first_phrase= nltk.pos_tag(nltk.word_tokenize(sentence[0:int(drug1_i[0])]))
      mid_phrase = nltk.pos_tag(nltk.word_tokenize(sentence[int(drug1_i[-1])+1:int(drug2_i[0])]))
      end_phrase = nltk.pos_tag(nltk.word_tokenize(sentence[int(drug2_i[-1])+1:]))
    except ValueError:
      print("error")
    before_count, middle_count, end_count = 0,0,0
    for tag in first_phrase:
        if tag[1].startswith('V'):
            before_count += 1
    for tag in mid_phrase:
        if tag[1].startswith('V'):
            middle_count += 1
    for tag in end_phrase:
        if tag[1].startswith('V'):
            end_count += 1
    return before_count, middle_count, end_count

def dist(idx, enum_list):
    '''helper function for trig_dist '''
    dif = [abs(idx-num) for num,word in enum_list]
    if len(dif) == 0:
        return 0
    else:
        return min(dif)
    
def trig_dist(x, row):
    '''counts distance from both drug mentions to closest trigger'''
    sentence = row['sentence_text']
    sentence = sentence.replace(row['drug1'], 'drug1')
    sentence = sentence.replace(row['drug2'], 'drug2')
    words = nltk.word_tokenize(sentence)
    
    try:
        drug1_i = words.index('drug1')
    except:
        drug1_i = 0
    try:
        drug2_i = words.index('drug2')
    except:
        drug2_i = 0
    enum_words = list(enumerate(words))
    enum_trigs = [(num,word) for num, word in enum_words if word in trigger_words]
    
    return dist(drug1_i, enum_trigs), dist(drug2_i, enum_trigs)

def subj_obj(x, row):
    '''binary feature that checks if drug1 is the nsubj or in its subtree 
    and if drug2 is dobj or in its subtree, returns 1 if both, 0 otherwise'''
    d1, d2 = row['drug1'],row['drug2']
    sent = row['sentence_text'].replace(d1,'drug1').replace(d2,'drug2')
    doc = nlp(sent)
    roots = [r for r in doc if r.head is r and r.pos == VERB] #and r.text in trigger_words]
    d1 = [d for d in doc if d.text == 'drug1']
    d2 = [d for d in doc if d.text == 'drug2']

    flag1 = False
    flag2 = False
    for r in roots:
        subj = [s for s in r.children if s.dep_[:5] == 'nsubj']
        obj = [o for o in r.children if o.dep_[:4] == 'dobj']
        for s in subj:
            if 'drug1' == s.text:
                flag1 = True
            for s1 in s.subtree:
                if s1.text == 'drug1':
                    flag1 = True
        for o in obj:
            if 'drug2' == o.text:
                flag2 = True
            for o1 in o.subtree:
                if o1.text == 'drug2':
                    flag2 = True
    if flag1 and flag2:
        return 1
    else:
        return 0
    
def trig_in_path(x, row):
    ''' binary feature if theres a trigger word in dep path to root '''
    d1, d2 = row['drug1'],row['drug2']
    
    sent = row['sentence_text'].replace(d1,'drug1').replace(d2,'drug2')
    doc = nlp(sent)
    #print("doc::::",doc)
    drugs = [d for d in doc if d.text == 'drug1' or d.text == 'drug2']
    #print("Printin drus : ",drugs)
    for d in drugs:
        #print("for loop for trig_path")
        #print("d befoe:",d)
        d = d.head
        #print("Printin drus d value after d.head: ",d)
        
        while d.head is not d:
            #print("entered while")
            if d.text in trigger_words:
               print("return 1")
               return 1
            else:
                #print("change -- d:",d)
                d = d.head
                #print("change -- d after:",d)
            if d.head==d :
                return 0
    return 0
    
def both_head(x, row):
    ''' checks if the head of both drug mentions is the same '''
    d1, d2 = row['drug1'],row['drug2']
    sent = row['sentence_text'].replace(d1,'drug1').replace(d2,'drug2')
    doc = nlp(sent)
    #roots = [r for r in doc if r.head is r and r.pos == VERB and r.text in trigger_words]
    drugs = [d for d in doc if d.text == 'drug1' or d.text == 'drug2']
    if  len(drugs) < 2:
        return 0
    if drugs[0].head == drugs[1].head: #and drugs[0].head.pos ==VERB:
        return 1
    return 0

def verb_vect(x, row):
    '''counts number of verbs in each segment '''
    
    sentence = row['sentence_text']
    words = nltk.word_tokenize(sentence)
    pos_words = nltk.pos_tag(words)
    
    try:
        drug1_i = words.index(row['drug1'])
        drug2_i = words.index(row['drug2'])
    except:
        return 0.0, 0.0, 0.0
   
    bef_verb = 0.0
    try:
        for i in range(drug1_i,0, -1):
            if pos_words[i][1].startswith('V') or pos_words[i][1].startswith('N'):
                bef_verb = np.mean(ddi_embedding_model[pos_words[i][0]])
                break
    except:
        bef_verb = 0.0
            
    bet_verb = 0.0
    try:
        for i in range(drug2_i,drug1_i, -1):
            if pos_words[i][1].startswith('V') or pos_words[i][1].startswith('N'):
                bet_verb = np.mean(ddi_embedding_model[pos_words[i][0]])
                break
    except:
        bet_verb = 0.0
            
            
    aft_verb = 0.0   
    try:
        for i in range(drug2_i, len(pos_words)):
            if pos_words[i][1].startswith('V') or pos_words[i][1].startswith('N'):
                aft_verb = np.mean(ddi_embedding_model[pos_words[i][0]])
                break
    except:
        aft_verb = 0.0

    return bef_verb, bet_verb, aft_verb


In [230]:
def build_features(df):
    c=0
    

    list_of_features = []
    negation_count_l = []
    trigger_count_l = []
    ultra_trigger_l = []
    subj_obj_l = []
    both_head_l = []
    trig_in_path_l = []
    for index, row in df.iterrows():
        negation_count_l.append(negation_count(index,row))
        trigger_count_l.append(trigger_count(index,row))
        ultra_trigger_l.append(ultra_trigger(index,row))
        subj_obj_l.append(subj_obj(index,row))
        both_head_l.append(both_head(index,row))
        trig_in_path_l.append(trig_in_path(index,row))
        
    list_of_features.append(negation_count_l)
    list_of_features.append(trigger_count_l)
    list_of_features.append(ultra_trigger_l)
    list_of_features.append(subj_obj_l)
    list_of_features.append(both_head_l)
    list_of_features.append(trig_in_path_l)

    drugs_between_drugs = []
    words_between_drugs = []
    for index, row in df.iterrows():
        drugs_between_drugs.append(num_drugs_between_drugs(index,row))
        words_between_drugs.append(num_words_between_drugs(index,row))
    list_of_features.append(drugs_between_drugs)
    list_of_features.append(words_between_drugs)
    
    #words_sep_by_and_l = []
    #key_phrase_l = []
    #for index, row in df.iterrows():
        #words_sep_by_and_l.append(words_sep_by_and(index,row))
        #key_phrase_l.append(key_phrase(index,row))
    #list_of_features.append(words_sep_by_and_l)
    #list_of_features.append(key_phrase_l)
    
    btg, betg, atg = [], [], []
    for index, row in df.iterrows():
        before, between, after = trigger_count2(index,row)
        btg.append(before)
        betg.append(between)
        atg.append(after)
    list_of_features.append(btg)
    list_of_features.append(betg)
    list_of_features.append(atg)
   
    n_btg, n_betg, n_atg = [], [], []
    for index, row in df.iterrows():
        before, between, after = negation_count2(index,row)
        n_btg.append(before)
        n_betg.append(between)
        n_atg.append(after)
    list_of_features.append(n_btg)
    list_of_features.append(n_betg)
    list_of_features.append(n_atg)
    
    v_btg, v_betg, v_atg = [], [], []
    for index, row in df.iterrows():
        before, between, after = verb_count(index,row)
        v_btg.append(before)
        v_betg.append(between)
        v_atg.append(after)
    list_of_features.append(v_btg)
    list_of_features.append(v_betg)
    list_of_features.append(v_atg)
    
    word2vec_b, word2vec_bet, word2vec_a = [], [], []
    for index, row in df.iterrows():
        verb_bef, verb_bet, verb_aft = verb_vect(index, row)
        word2vec_b.append(verb_bef)
        word2vec_bet.append(verb_bet)
        word2vec_a.append(verb_aft)
        
    list_of_features.append(word2vec_b)
    list_of_features.append(word2vec_bet)
    list_of_features.append(word2vec_a)

    print("List_of_features",list_of_features)
    return list_of_features

In [231]:
#read
vec = TfidfVectorizer(ngram_range=(1, 2), max_df=0.8)

arr_train_feature_sparse = vec.fit_transform(df_train['sentence_text'])
print("arr_train_feature_sparse",arr_train_feature_sparse)
arr_train_feature = arr_train_feature_sparse.toarray()
print("arr_train_feature",arr_train_feature)


arr_test_feature_sparse = vec.transform(df_test["sentence_text"])
arr_test_feature = arr_test_feature_sparse.toarray()

arr_test_feature_sparse1 = vec.transform(idata["sentence_text"])
arr_test_feature1 = arr_test_feature_sparse1.toarray()

arr_train_feature_sparse   (0, 53)	0.14827562075373057
  (0, 164)	0.14827562075373057
  (0, 104)	0.14827562075373057
  (0, 113)	0.14827562075373057
  (0, 108)	0.12450206747763227
  (0, 387)	0.14827562075373057
  (0, 13)	0.14827562075373057
  (0, 268)	0.14827562075373057
  (0, 331)	0.14827562075373057
  (0, 144)	0.14827562075373057
  (0, 305)	0.14827562075373057
  (0, 180)	0.14827562075373057
  (0, 343)	0.14827562075373057
  (0, 425)	0.14827562075373057
  (0, 329)	0.14827562075373057
  (0, 149)	0.14827562075373057
  (0, 160)	0.14827562075373057
  (0, 315)	0.14827562075373057
  (0, 57)	0.14827562075373057
  (0, 421)	0.14827562075373057
  (0, 406)	0.14827562075373057
  (0, 209)	0.14827562075373057
  (0, 165)	0.14827562075373057
  (0, 418)	0.14827562075373057
  (0, 237)	0.14827562075373057
  :	:
  (49, 251)	0.15618592291116395
  (49, 419)	0.15618592291116395
  (49, 81)	0.15618592291116395
  (49, 426)	0.15618592291116395
  (49, 269)	0.15618592291116395
  (49, 232)	0.15618592291116395
  (49,

In [232]:
def convert_to_numpy(list_):
    x = np.asarray(list_)
    y = x.reshape(-1, 1)
    return y

def create_feature_set(mainset,features):
    count=0
    for list_f in features:
        count=count+1
        print("Feature loading:")
        print(count)
        mainset = np.append(mainset, convert_to_numpy(list_f), axis=1)
    return mainset

In [233]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [234]:
print(idata['sentence_text'])

0    Pharmacokinetic studies indicate that administ...
1    Co-administration of naltrexone with Acamprosa...
Name: sentence_text, dtype: object


In [235]:
train_feature_list = build_features(df_train)
print("arr_train_feature:",arr_train_feature)
#print("train_feature_list:",train_feature_list)
train_feature_set = create_feature_set(arr_train_feature, train_feature_list)
#print("Train_feature_set:",train_feature_set)
test_feature_list = build_features(df_test)
test_feature_set = create_feature_set(arr_test_feature, test_feature_list)

test_feature_list1 = build_features(idata)
test_feature_set1 = create_feature_set(arr_test_feature1, test_feature_list1)

List_of_features [[0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [2, 1, 6, 5, 2, 4, 0, 1, 6, 0, 3, 0, 7, 1, 4, 0, 5, 6, 3, 0, 0, 0, 1, 

In [236]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
mn = MultinomialNB()
print("Baseline with only vectorized bag of words:")

print("starting to fit")
mn_model = mn.fit(arr_train_feature, df_train.ddi_label)

print("starting to predict")
pred_test = mn_model.predict(arr_test_feature)
#print("test_labels:prediction:",str(df_test.ddi_label),":",pred_test)
#pred_test1 = mn_model.predict(arr_test_feature1)
#print("input_pred:",pred_test1)

print('accuracy: ', accuracy_score(df_test.ddi_label, pred_test))
print(f1_score(df_test.ddi_label, pred_test, average=None, pos_label = 1))

Baseline with only vectorized bag of words:
starting to fit
starting to predict
accuracy:  0.8755980861244019
[0.93367347 0.        ]


In [237]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV

def run_model(model):
    print("starting to fit")
    fit_model = model.fit(train_feature_set, df_train.ddi_label)
    print("starting to predict")
    pred_test = fit_model.predict(test_feature_set)
    #input
    '''pred_test1 = fit_model.predict(test_feature_set1)
    print("input_pred:",pred_test1)'''

    print("df_test.ddi_label:",df_test.ddi_label)
    print("pred:",pred_test)
    print('accuracy: ', accuracy_score(df_test.ddi_label, pred_test))
    print(f1_score(df_test.ddi_label, pred_test, average=None, pos_label = 1))
    pred_test = fit_model.predict(test_feature_set1)
    print("predicions:",pred_test)

In [238]:
knn = KNeighborsClassifier(n_neighbors=3)
run_model(knn)

starting to fit
starting to predict
df_test.ddi_label: 0      0
1      0
2      0
3      0
4      0
      ..
219    0
220    0
221    1
222    0
223    0
Name: ddi_label, Length: 209, dtype: int64
pred: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
accuracy:  0.8708133971291866
[0.92987013 0.18181818]
predicions: [0 1]


In [239]:
import pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(knn,file)


In [240]:

with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
#knn = KNeighborsClassifier(n_neighbors=3)
#run_model(pickle_model)
print("label:",test_feature_set1)

#fit_model = pickle_model.fit(test_feature_set1, df_test.ddi_label)
print("starting to predict")
pred_test1 = pickle_model.predict(test_feature_set1)
print(pred_test1)


label: [[ 0.          0.          0.          0.          0.          0.
   0.14789091  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.16148233  0.169819    0.          0.          0.          0.
   0.          0.1917471   0.1917471   0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   

In [241]:

with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)
knn = KNeighborsClassifier(n_neighbors=3)
run_model(knn)

from sklearn.preprocessing import StandardScaler

scaler =StandardScaler()
# Calculate the accuracy score and predict target values
Ys = scaler.fit_transform(pd.DataFrame(df_train['sentence_text']))
Yt = scaler.transform(pd.DataFrame(df_test['sentence_text']))

Ys=Ys.reshape(-1,1)
Yt=Yt.reshape(-1,1)
# Ys=vect.fit_transform(Ys)
# Yt=vect.transform(Yt)
print(Yt.shape)
print(Ys.shape)
Yte = scaler.transform(pd.DataFrame(idata['sentence_text']))
Yte=Yte.reshape(-1,1)
# score = pickle_model.score(Ys,Yt)
# print("Test score: {0:.2f} %".format(100 * score))

 


Ypredict = pickle_model.predict(Yte)


starting to fit
starting to predict
df_test.ddi_label: 0      0
1      0
2      0
3      0
4      0
      ..
219    0
220    0
221    1
222    0
223    0
Name: ddi_label, Length: 209, dtype: int64
pred: [0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
accuracy:  0.8708133971291866
[0.92987013 0.18181818]
predicions: [0 1]


ValueError: ignored